In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = ' --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell'

In [ ]:
session = (
    SparkSession
    .builder
    .master("local[2]")
    .appName("Driver")
    .getOrCreate()
)
session.sparkContext.setLogLevel('WARN')

In [ ]:
session.stop()

In [ ]:
source = (session.readStream
          .format("kafka")
          .option('kafka.bootstrap.servers', 'localhost:9092')
          .option('subscribe', 'stream_topic')
          ).load()

In [ ]:
df = (source
      .selectExpr('CAST(value AS STRING)', 'offset'))

In [ ]:
console = (df
           .writeStream
           .format('console'))

In [ ]:
console.start().awaitTermination()

In [ ]:
writer_to_csv = (
    df
    .writeStream
    .format("csv")
    .option("path", "./data")
)

In [ ]:
session.conf.set("spark.sql.streaming.checkpointLocation","./checkPoint")
writer_to_csv.start().awaitTermination()

In [11]:
accum = session.sparkContext.accumulator(0)
foreach_data = (
    df
    .writeStream
    .foreach(lambda x:accum.add(1))
    .start()
    .awaitTermination()
)

22/09/06 19:44:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 522
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 3935|  3943|
+----------------+------+

-------------------------------------------
Batch: 523
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 3936|  3944|
+----------------+------+

-------------------------------------------
Batch: 524
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 3937|  3945|
+----------------+------+

-------------------------------------------
Batch: 525
-------------------------------------------
+----------------+------+
|           value|offset|
+--------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/dmitryrusack/PycharmProjects/SparkArticle/venv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dmitryrusack/PycharmProjects/SparkArticle/venv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [12]:
accum.value

-------------------------------------------
Batch: 545
-------------------------------------------


20

In [17]:
session.stop()

22/09/06 19:50:06 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@4b27a8bc is aborting.
22/09/06 19:50:06 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@4b27a8bc aborted.
22/09/06 19:50:06 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@288285e5 is aborting.
22/09/06 19:50:06 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@288285e5 aborted.
22/09/06 19:50:06 ERROR MicroBatchExecution: Query [id = f2e24efc-2109-4580-84f2-10a81398a8c6, runId = 2252c41b-d062-46bb-865a-f12f5ae1e687] terminated with error
org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:749)
	at org.apache.spark.sql.execution.

Exception in thread "stream execution thread for [id = af12f3a4-3438-40e0-9eb3-6017913cd973, runId = 8d3d55a8-813a-46a6-a7cf-9cee2276c071]" Exception in thread "stream execution thread for [id = f2e24efc-2109-4580-84f2-10a81398a8c6, runId = 2252c41b-d062-46bb-865a-f12f5ae1e687]" org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef.deactivateInstances(StateStoreCoordinator.scala:119)
	at org.apache.spark.sql.streaming.StreamingQueryManager.notifyQueryTermination(StreamingQueryManager.scala:406)
	at org.apache.spark.sql.execution.streaming.StreamExecution.$anonfun$runStream$3(StreamExecution.scala:357)
	at scala.ru

22/09/06 19:50:06 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.RejectedExecutionException: Task java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask@1855fc60[Not completed, task = java.util.concurrent.Executors$RunnableAdapter@665a8311[Wrapped task = org.apache.spark.rpc.netty.NettyRpcEnv$$anon$1@49fd0a2f]] rejected from java.util.concurrent.ScheduledThreadPoolExecutor@32ec82a2[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0]
